<br>
    <center>МИНИСТЕРСТВО НАУКИ И ВЫСШЕГО ОБРАЗОВАНИЯ</center>
    <center>федеральное государственное автономное образовательное учреждение высшего образования </center> <center>«Самарский национальный исследовательский университет имени академика С.П. Королева»</center>
    <center>(Самарский университет)</center> </br>

<br/>

<br>
<center>Институт 	     информатики и кибернетики</center>                                                   	  
<center>Кафедра 	     технической кибернетики</center>                                                              	
</br>
<br/>

<br>

<br/>
<br/>
<br/>
<br>
<center>ОТЧЕТ</center>
<center>по лабораторной работе №1</center>

<center>«Введение в MapReduce модель на Python»</center>
<br/>
<center>по дисциплине <strong>«Большие данные»</strong></center>
<br/>
<center></center>
</br>
<br/>
<br/>
<br/>

<br>
<p style="text-align:right;">Обучающийся: Скляров Д.В.
<br>6131-010402D
<br>    
<br>Преподаватель: Попов С.Б.
</p>
<br/>
<br/>
<br/>
    <br/>
<br/>
<br/>
<center>Самара 2024</center>


## Спецификация MapReduce

```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```


In [13]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce Distributed

```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```


In [81]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for (k2, v2) in map_partition:
            p = partitions[PARTITIONER(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]


def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers


def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    map_partitions = map(lambda record_reader: flatten(
        map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
    if COMBINER != None:
        map_partitions = map(lambda map_partition: flatten(
            map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
    reduce_partitions = groupbykey_distributed(
        map_partitions, PARTITIONER)  # shuffle
    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(
        lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (
        k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
    return reduce_outputs

# Упражнения

Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.


### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.


In [48]:
from typing import Iterator


input_vals = [1, 5, 5, 4, 100]


def RECORDREADER():
    for idx, val in enumerate(input_vals):
        yield (idx, val)


def MAP(el_id: int, val: int):
    yield (0, val)


def REDUCE(key: int, vals: Iterator[int]):
    max_value = 0
    for v in vals:
        if max_value < v:
            max_value = v
    yield (max_value)


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[100]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [52]:
from typing import Iterator


input_vals = [1, 6, 5, 5, 4, 100]


def RECORDREADER():
    for idx, val in enumerate(input_vals):
        yield (idx, val)


def MAP(el_id: int, val: int):
    yield (0, val)


def REDUCE(k, rows: Iterator[int]):
    sum = 0
    count = 0
    for val in rows:
        sum += val
        count += 1
    if count > 0:
        yield (sum / count)


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[20.166666666666668]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах


In [80]:
from typing import NamedTuple
from typing import Iterator


class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str


input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]


def MAP(_, row: User):
    yield (row.gender, row)


def REDUCE(key: str, rows: Iterator[NamedTuple]):
    aggregate = 0
    for _ in rows:
        aggregate += 1
    yield (key, aggregate)


def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('male', 1), ('female', 3)]

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов


In [91]:
from typing import Iterator
import numpy as np

d1 = "str1"
d2 = "str2"
d3 = "str3"
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2


def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for (docid, document) in enumerate(split):
            for (lineid, line) in enumerate(document.split('\n')):
                yield ("{}:{}".format(docid, lineid), line)

    split_size = int(np.ceil(len(documents)/maps))
    for i in range(0, len(documents), split_size):
        yield RECORDREADER(documents[i:i+split_size])


def MAP(object_id: str, object: str):
    yield (object, 1)


def REDUCE(word: str, counts: Iterator[int]):
    resulting = None
    for c in counts:
        resulting = (word)
        break
    yield resulting


# try to set COMBINER=REDUCE and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(
    INPUTFORMAT,
    MAP,
    REDUCE,
    COMBINER=None)
partitioned_output = [(partition_id, list(partition))
                      for (partition_id, partition) in partitioned_output]
partitioned_output

6 key-value pairs were sent over a network.


[(0, ['str2', 'str3']), (1, ['str1'])]

## Операторы реляционной алгебры

### Selection (Выборка)

**The Map Function**: Для каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.


In [102]:
from typing import Iterator


input_vals = [1, 6, 5, 5, 4, 100]


def RECORDREADER():
    for idx, val in enumerate(input_vals):
        yield (idx, val)


def MAP(el_id: int, val: int):
    if val <= 5:
        yield (val, val)


def REDUCE(k, rows: Iterator[int]):
    yield k


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[1, 5, 4]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая из $t$ те значения, атрибуты которых не принадлежат $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара $(t′, t′)$.


In [116]:
from typing import Iterator


S = [1, 6, 5, 5, 4, 100]
ts = [
    (1, 6, 3),
    (5, 7, 2),
    (9, 3, 4),
    (1, 10, 11)
]


def RECORDREADER():
    for idx, val in enumerate(ts):
        yield (idx, val)


def MAP(el_id: int, val: int):
    global S
    for elem in val:
        if elem in S:
            yield (elem, elem)


def REDUCE(k, rows: Iterator[int]):
    yield k, k


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(1, 1), (6, 6), (5, 5), (4, 4)]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.


In [140]:
from typing import Iterator

ts = [
    (1,),
    (5,),
    (9, 10),
    (1, 11),
]


def RECORDREADER():
    for idx, val in enumerate(ts):
        yield (idx, val)


def MAP(el_id: int, val: int):
    yield (val, val)


def REDUCE(k, rows: Iterator[int]):
    for key in k:
        yield (key, rows)


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(1, [(1,)]),
 (5, [(5,)]),
 (9, [(9, 10)]),
 (10, [(9, 10)]),
 (1, [(1, 11)]),
 (11, [(1, 11)])]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.


In [143]:
from typing import Iterator

ts = [
    (1, 11),
    (5, 2),
    (9, 10),
    (1, 11),
]


def RECORDREADER():
    for idx, val in enumerate(ts):
        yield (idx, val)


def MAP(el_id: int, val: int):
    yield (val, val)


def REDUCE(k, rows: Iterator[int]):
    for key in k:
        if len(rows) == 2:
            yield (key, key)


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(1, 1), (11, 11)]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.


In [155]:
1 << 0

1

In [176]:
from typing import Iterator

ts = [
    (1, 2, 3, 4, 5, 6),
    (5, 6, 7, 8, 1, 2)
]
R = 1 << 0
S = 1 << 1


def RECORDREADER():
    for series_id, series in enumerate(ts):
        for value in series:
            yield (series_id, value)


def MAP(series_id: int, val: int):
    yield (val, 1 << series_id)


def REDUCE(k, id: Iterator[int]):
    for i in id:
        if i == R:
            yield (k, k)


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6)]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.


In [194]:
from typing import Iterator

R = (
    (1, 2, 3, 4, 5, 6),
    (5, 6, 7, 8, 1, 2)
)
S = (
    (4, 1, 5, 2, 1, 5),
    (11, 21, 12, 31, 4),
)
SERIES = [R, S]


def RECORDREADER():
    for series_id, series in enumerate(SERIES):
        yield (series_id, series)


def MAP(series_id: int, val: int):
    yield (val[1], (series_id, val[0]))


def REDUCE(k, par: Iterator[int]):
    for p in par:
        _ = p[0]
        for vals in p[1]:
            yield (vals)


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[1, 2, 3, 4, 5, 6, 4, 1, 5, 2, 1, 5]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.


In [211]:
from typing import Iterator


SERIES = [
    (
        (1, 20, 3),
        (4, 5, 100),
        (10, 10, 30),
    ),
    (
        (12, 20, 3),
        (42, 35, -1),
        (10, 10, 30),
    )
]


def RECORDREADER():
    for series_id, series in enumerate(SERIES):
        yield (series_id, series)


def MAP(series_id: int, val: int):
    yield (val[0], val[1])


def REDUCE(k, par: Iterator[int]):
    yield (k, max(par[0]))


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[((1, 20, 3), 100), ((12, 20, 3), 42)]

## Вычисление TF-IDF (Term Frequency – Inverse Document Fraquency)

Реализуется в три этапа:

**Этап 1:** Частота слова в документе

**Этап 2:** Количество документов, в которых встречается слово

**Этап 3:** Расчёт TF-IDF


In [214]:
DOCS = {
    'An empirical analysis of stateful operator migration for online scheduling in distributed stream processing systems':
    """
    Streaming data is the data from sensors as well as other real-time surveillance systems. Distributed stream processing systems are the software that manages such data. Such frameworks have to deliver outcomes on the go instantly. They are susceptible to delay and malfunction or system failures. The system must be tolerant of faults and always accessible. Many variables, such as improved network arrival rates, node failures, and so on, disrupt the system's reliability. Some operators need to be relocated online from one physical resource to another to manage or reimburse a slow or failing node. In this study, we propose a co-location based systematic migration heuristic for live operator migration between physical resources using a migration map revised with costs for each migration. The suggested method evaluates continuous operator performance patterns and makes online scheduling decisions based on the same. The decisions include migrating operators during a node failure or straggling.
    """,
    'Theodolite: Scalability Benchmarking of Distributed Stream Processing Engines in Microservice Architectures':
    """
    Distributed stream processing engines are designed with a focus on scalability to process big data volumes in a continuous manner. We present the Theodolite method for benchmarking the scalability of distributed stream processing engines. Core of this method is the definition of use cases that microservices implementing stream processing have to fulfill. For each use case, our method identifies relevant workload dimensions that might affect the scalability of a use case. We propose to design one benchmark per use case and relevant workload dimension. We present a general benchmarking framework, which can be applied to execute the individual benchmarks for a given use case and workload dimension. Our framework executes an implementation of the use case's dataflow architecture for different workloads of the given dimension and various numbers of processing instances. This way, it identifies how resources demand evolves with increasing workloads. Within the scope of this paper, we present 4 identified use cases, derived from processing Industrial Internet of Things data, and 7 corresponding workload dimensions. We provide implementations of 4 benchmarks with Kafka Streams and Apache Flink as well as an implementation of our benchmarking framework to execute scalability benchmarks in cloud environments. We use both for evaluating the Theodolite method and for benchmarking Kafka Streams' and Flink's scalability for different deployment options.
    """,
    'A hybrid distributed batch-stream processing approach for anomaly detection':
    """
    Batch and stream processing are separately and efficiently applied in many applications. However, some newer data-driven applications such as the Internet of Things and cloud computing call for hybrid processing approaches in order to handle the speed and accuracy required for processing such complex data. In this paper, we propose a Hybrid Distributed Batch-Stream (HDBS) architecture for anomaly detection in real-time data. The hybrid architecture, while benefiting from the accuracy provided by batch processing, also enjoys the speed and real-time features of stream processing. In the proposed architecture, our focus is on the algorithmic aspects of hybrid processing including the interaction models between batch and stream processing units, the characteristics of batch and stream machine learning algorithms and the principles of merging the results of different processing units. The driving idea of such combination is that the results of batch and stream processing units are complementary with each other, as one of them constructs accurate models based on previous data, and the other one is capable of processing new stream data in real-time. Furthermore, we propose a generalized version of the HDBS with respect to its algorithms and communication policy levels. In the generalized HDBS architecture, we address the various aspects of the interaction between the batch and stream processing units, and the merging operations to produce the final results. the evaluations of the proposed architecture using various criteria (accuracy, space complexity, and time complexity) demonstrate that the accuracy of the proposed method is higher than the accuracy of the batch processing methods, its time complexity is also similar to one of the stream processing methods and much less than the batch processing methods, which makes our proposed architecture an efficient and practical solution for real-time anomaly detection.
    """,
    'Topology-aware task allocation for online distributed stream processing applications with latency constraints':
    """
    There have been increasing demands for real time processing of the ever-growing data. In order to meet this requirement and ensure the reliable processing of streaming data, a variety of distributed stream processing architectures and platforms have been developed, which handles the fundamental task of allocating processing tasks to the currently available physical resources and routing streaming data between these resources. However, many stream processing systems lack an intelligent scheduling mechanism, in which their default schedulers allocate tasks without taking resource demands and availability, or the transfer latency between resources into consideration. Besides, stream processing has a strict request for latency. Thus it is important to give latency guarantee for distributed stream processing. In this paper, we propose two new algorithms for stream processing with latency guarantee, both the algorithms consider transfer latency and resource demand in task allocation. Both algorithms can guarantee latency constraints. Algorithm AHA reduces more than 21.3% and 58.9% resources compared with the greedy and the round-robin algorithms, and algorithm PHA further improves the resource utilization to 32.1% and 73.2%.
    """,
    'Improvement Design for Distributed Real-Time Stream Processing Systems':
    """
    In the era of Big Data, typical architecture of distributed real-time stream processing systems is the combination of Flume, Kafka, and Storm. As a kind of distributed message system, Kafka has the characteristics of horizontal scalability and high throughput, which is manly deployed in many areas in order to address the problem of speed mismatch between message producers and consumers. When using Kafka, we need to quickly receive data sent by producers. In addition, we need to send data to consumers quickly. Therefore, the performance of Kafka is of critical importance to the performance of the whole stream processing system. In this paper, we propose the improved design of real-time stream processing systems, and focus on improving the Kafka’s data loading process. We use Kafka cat to transfer data from the source to Kafka topic directly, which can reduce the network transmission. We also utilize the memory file system to accelerate the process of data loading, which can address the bottleneck and performance problems caused by disk I/O. Extensive experiments are conducted to evaluate the performance, which show the superiority of our improved design.
    """,
    'Comprehensive analysis of the heterogeneous computing performance of DNNs under typical frameworks on cloud and edgecomputing platforms':
    """
    In this paper, nearly 40 commonly used deep neural network(DNN) models are selected, and their cross-platform and cross-inference frameworks are deeply analysed. The main metrics of accuracy, the total number of model parameters, the computational complexity, the accuracy density, the inference time, the memory consumption and other related parameters are used to measure their performance. The heterogeneous computing experiment is implemented on both the Google Colab cloud computing platform and the Jetson Nano embedded edge computing platform. The obtained performance is compared with that of two previous computing platforms: a workstation equipped with an NVIDIA Titan X Pascal and an embedded system based on an NVIDIA Jetson TX1 board. In addition, on the Jetson Nano embedded edge computing platform, different inference frameworks are investigated to evaluate the inference efficiency of the DNN models. Regression models are established to characterize the variation in the computing performance of different DNN classification algorithms so that the inference results of unknown models can be estimated. ANOVA methods are proposed to quantify the differences between models. The experimental results have important guiding significance for the better selection, deployment and application of DNN models in practice. Codes are available at this https URL https://github.com/Foreverzfy/Model-Test.
    """,
    'Low-power deep learning edge computing platform for resource constrained lightweight compact UAVs':
    """
    Unmanned Aerial Vehicles (UAVs), which can operate autonomously in dynamic and complex environments, are becoming increasingly common. Deep learning techniques for motion control have recently taken a major qualitative step since vision-based inference tasks can be executed directly on edge. The goal is to fully integrate the machine learning (ML) element into small UAVs. However, given the limited payload capacity and energy available on small UAVs, integrating computing resources sufficient to host ML and vehicle control functions is still challenging. This paper presents a modular and generic system that can control the UAV by evaluating vision-based ML tasks directly inside the resource-constrained UAV. Two different vision-based navigation configurations were tested and demonstrated. The first configuration implements an autonomous landing site detection system, tested with two models based on LeNet-5 and MobileNetV2, respectively. This allows the UAV to change its planned path accordingly and approach the target to land. Moreover, a model for people detection based on a custom MobileNetV2 network was evaluated in the second configuration. Finally, the execution time and power consumption were measured and compared with a cloud computing approach. The results show the ability of the developed system to dynamically react to the environment to provide the necessary maneuver after detecting the target exploiting only the constrained computational resources of the UAV controller. Furthermore, we demonstrated that moving to the edge, instead of using cloud computing inference, decreases the energy requirement of the system without reducing the quality of service.
    """,
    'Heterogeneous edge computing open platforms and tools for internet of things':
    """
    With the continuous development of Internet of Things (IoT) and the overwhelming explosion of Big Data, edge computing serves as an efficient computing mode for time stringent data processing, which can bypass the constraints of network bandwidth and delay, and has been one of the foundation of interconnected applications. Although edge computing has gradually become one of bridges between cloud computing centers and mobile terminals, the literature still lacks a thorough review on the recent advances in edge computing platforms. In this paper, we firstly introduce the definition of edge computing and advantages of edge computing platform. And then, we summarize the key technologies of constructing an edge computing platform, and propose a general framework for edge computing platform. The role of distributed storage management systems in building edge computing platform is elaborated in detail. Furthermore, we give some applications to illustrate how to use third-party edge computing platforms to build specific applications. Finally, we briefly outline current open issues of edge computing platform based on our literature survey.
    """
}

In [238]:
from typing import Iterator

SN = {}


def RECORDREADER1():
    for docname, contents in DOCS.items():
        yield (docname, contents)


def MAP1(docname: int, contents: int):
    for word in contents.split(' '):
        try:
            SN[docname] += 1
        except KeyError:
            SN[docname] = 1
        yield ((word, docname), 1)


def REDUCE1(key, val):
    word, docname = key
    tn = 0
    for a in val:
        tn += a
    yield ((word, docname), tn / SN[docname])


output1 = MapReduce(RECORDREADER1, MAP1, REDUCE1)
output1 = list(output1)
output

[(('\n',
   'An empirical analysis of stateful operator migration for online scheduling in\xa0distributed\xa0stream\xa0processing\xa0systems'),
  0.006329113924050633),
 (('',
   'An empirical analysis of stateful operator migration for online scheduling in\xa0distributed\xa0stream\xa0processing\xa0systems'),
  0.04430379746835443),
 (('Streaming',
   'An empirical analysis of stateful operator migration for online scheduling in\xa0distributed\xa0stream\xa0processing\xa0systems'),
  0.006329113924050633),
 (('data',
   'An empirical analysis of stateful operator migration for online scheduling in\xa0distributed\xa0stream\xa0processing\xa0systems'),
  0.012658227848101266),
 (('is',
   'An empirical analysis of stateful operator migration for online scheduling in\xa0distributed\xa0stream\xa0processing\xa0systems'),
  0.006329113924050633),
 (('the',
   'An empirical analysis of stateful operator migration for online scheduling in\xa0distributed\xa0stream\xa0processing\xa0systems'),
  0.

In [242]:
def RECORDREADER2():
    for bypass in output1:
        yield bypass


def MAP2(par: tuple[str, str], tf: int):
    word, dockname = par
    yield (word, (dockname, tf, 1))


def REDUCE2(key, rows):
    n = 0
    word = key,
    dockname = None
    tf = None
    for row in rows:
        dockname, tf, ammount = row
        n += ammount
    if dockname is not None:
        yield ((word, dockname), (tf, n))


output2 = MapReduce(RECORDREADER2, MAP2, REDUCE2)
output2 = list(output2)
output2

[((('\n',),
   'Heterogeneous\xa0edge\xa0computing\xa0open\xa0platforms\xa0and tools for internet of things'),
  (0.005649717514124294, 8)),
 ((('',),
   'Heterogeneous\xa0edge\xa0computing\xa0open\xa0platforms\xa0and tools for internet of things'),
  (0.03954802259887006, 8)),
 ((('Streaming',),
   'An empirical analysis of stateful operator migration for online scheduling in\xa0distributed\xa0stream\xa0processing\xa0systems'),
  (0.006329113924050633, 1)),
 ((('data',),
   'Heterogeneous\xa0edge\xa0computing\xa0open\xa0platforms\xa0and tools for internet of things'),
  (0.005649717514124294, 6)),
 ((('is',),
   'Heterogeneous\xa0edge\xa0computing\xa0open\xa0platforms\xa0and tools for internet of things'),
  (0.005649717514124294, 8)),
 ((('the',),
   'Heterogeneous\xa0edge\xa0computing\xa0open\xa0platforms\xa0and tools for internet of things'),
  (0.04519774011299435, 8)),
 ((('from',),
   'Improvement Design for\xa0Distributed\xa0Real-Time\xa0Stream\xa0Processing\xa0Systems'),
  (0.

In [246]:
import math


def RECORDREADER3():
    for bypass in output2:
        yield bypass


def MAP3(par: tuple[str, str], tfn: int):
    word, dockname = par
    tf, n = tfn
    yield ((word, dockname), tf * math.log10(len(SN.items()) / n))


def REDUCE3(par, rows):
    yield par, rows


output3 = MapReduce(RECORDREADER3, MAP3, REDUCE3)
output3 = list(output3)
output3

[((('\n',),
   'Heterogeneous\xa0edge\xa0computing\xa0open\xa0platforms\xa0and tools for internet of things'),
  [0.0]),
 ((('',),
   'Heterogeneous\xa0edge\xa0computing\xa0open\xa0platforms\xa0and tools for internet of things'),
  [0.0]),
 ((('Streaming',),
   'An empirical analysis of stateful operator migration for online scheduling in\xa0distributed\xa0stream\xa0processing\xa0systems'),
  [0.005715759411341415]),
 ((('data',),
   'Heterogeneous\xa0edge\xa0computing\xa0open\xa0platforms\xa0and tools for internet of things'),
  [0.0007058685684084742]),
 ((('is',),
   'Heterogeneous\xa0edge\xa0computing\xa0open\xa0platforms\xa0and tools for internet of things'),
  [0.0]),
 ((('the',),
   'Heterogeneous\xa0edge\xa0computing\xa0open\xa0platforms\xa0and tools for internet of things'),
  [0.0]),
 ((('from',),
   'Improvement Design for\xa0Distributed\xa0Real-Time\xa0Stream\xa0Processing\xa0Systems'),
  [0.001567864560749902]),
 ((('sensors',),
   'An empirical analysis of stateful operat